In [1854]:
import re
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from tabulate import tabulate

In [1855]:
superRoot = "../outputs"

In [1856]:
def extractNumberFromFilename(filename):
    match = re.search(r'\d+', filename)  # Use regular expression to find numeric part
    if match:
        return int(match.group())  # Convert the matched part to an integer
    else:
        return 'N/A'

# Define a function to extract the number after ":"
def extractNumberFromLine(line):
    tokens = line.split(":")
    if len(tokens) >= 2:
        try:
            afterPoints = tokens[1].strip()
            num = afterPoints.split(" ")[0]
            return float(num)
        except ValueError:
            return 'N/A'
    return 'N/A'

# Define a function to process each file
def processFile(filePath):
    with open(filePath, 'r') as file:
        lines = file.readlines()
        numNodes = extractNumberFromFilename(os.path.splitext(os.path.basename(filePath))[0])  # Extract the number from the file name
        time, memory, cost, cuts, leafReached = 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'
        
        for i, line in enumerate(lines):
            if i == 3 and line.startswith("Time"):
                time = extractNumberFromLine(line)
            elif i == 4 and line.startswith("Memory"):
                memory = extractNumberFromLine(line)
            elif i == 5 and line.startswith("Best"):
                inf = line.split(":")[1]
                if inf == 'inf':
                    cost = float('inf')
                else:
                    cost = extractNumberFromLine(line)
            elif i == 6 and (line.startswith("Cuts") or line.startswith("Cut")):
                cuts = extractNumberFromLine(line)
            elif i == 7 and line.startswith("Leaf"):
                if "Leaf node was reached" in line:
                    leafReached = True
                elif "Leaf node was NOT reached" in line:
                    leafReached = False
            elif i > 8:
                break

        pattern = r"^(.*?)\.txt$"
        name = re.match(pattern, os.path.basename(filePath)).group(1)
        
        return {
            'name': name,
            'numNodes': numNodes,
            'time': time,
            'memory': memory,
            'cost': cost,
            'cuts': cuts,
            'leafReached': leafReached
        }


In [1857]:
# Define a function to process a root directory and return a DataFrame
def processDirectory(root):
    data = []
    
    for filename in os.listdir(root):
        filePath = os.path.join(root, filename)
        if os.path.isfile(filePath) and filename.endswith(".txt"):
            fileData = processFile(filePath)
            data.append(fileData)
    
    # Create a DataFrame for the current directory's data
    algoType = os.path.basename(root)
    df = pd.DataFrame(data)
    df['algoType'] = algoType
    
    return df

# Define the algoOutputProcessor function
def algoOutputProcessor(superRoot):
    allDfs = []

    for root, dirs, files in os.walk(superRoot):
        if any(filename.endswith(".txt") for filename in files):
            df = processDirectory(root)
            allDfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    algoOutputs = pd.concat(allDfs, ignore_index=True)

    return algoOutputs

In [1858]:
results_df = algoOutputProcessor(superRoot)
pd.set_option('display.max_rows', 10)
results_df

,name,numNodes,time,memory,cost,cuts,leafReached,algoType
0,d493,493,0.799885,8.0,45427.088847,N/A,N/A,twiceAroundTheTree
1,d1655,1655,10.241591,8.0,85681.078417,N/A,N/A,twiceAroundTheTree
2,rl1323,1323,6.204457,8.0,380421.587604,N/A,N/A,twiceAroundTheTree
3,pr1002,1002,3.276209,8.0,342244.462396,N/A,N/A,twiceAroundTheTree
4,fl1400,1400,6.625004,8.0,27915.523477,N/A,N/A,twiceAroundTheTree
...,...,...,...,...,...,...,...,...
186,rat99,99,1800.010000,0.0,2130.060000,7783735.0,False,BnB_trivialSolution_cpp
187,ts225,225,1800.040000,0.0,276532.000000,2324898.0,False,BnB_trivialSolution_cpp
188,pr152,152,1800.040000,0.0,160980.000000,3166609.0,False,BnB_trivialSolution_cpp
189,u159,159,1800.090000,0.0,43375.700000,3120551.0,False,BnB_trivialSolution_cpp


In [1859]:
table = tabulate(results_df.sort_values(by='numNodes').reset_index(), headers='keys', tablefmt='psql')

with open("./tables/table.csv", 'w') as output_file:
    output_file.write(table)

In [1860]:
# Define a dictionary to map algoType to colors
algoColors = {
    'BnB_original_cpp': 'blue',
    'BnB_trivialSolution_cpp': 'green',
    'twiceAroundTheTree': 'red',
    'christofides': 'purple'
}

# Extract numNodes, timeMinutes, and algoType columns
numNodes = results_df['numNodes']
timeSeconds = results_df['time']
algoType = results_df['algoType']

# Convert time from seconds to minutes
timeMinutes = timeSeconds / 60.0  # 1 minute = 60 seconds

# Create a color map for different algoType values
uniqueAlgoTypes = algoType.unique()
colorMap = plt.get_cmap('viridis', len(uniqueAlgoTypes))

fig, ax = plt.subplots(figsize=(6, 5))
for algo, color in algoColors.items():
    mask = algoType == algo
    ax.scatter(numNodes[mask], timeMinutes[mask], label=algo, s=30, c=color, alpha=0.65)

    # Check the algorithm type and fit the appropriate model
    if algo == 'christofides':
        # Fit a cubic model for Christofides Algorithm
        coefficients = np.polyfit(numNodes[mask], timeMinutes[mask], 3)
        polynomial = np.poly1d(coefficients)
        x_range = np.linspace(min(numNodes[mask]), max(numNodes[mask]), 100)
        y_range = polynomial(x_range)
        ax.plot(x_range, y_range, label='Christofides Trend', color=color, alpha=0.2, linewidth=2)

    elif algo == 'twiceAroundTheTree':
        # Define the model function for Twice Around the Tree complexity
        def model_func(numNodes, a, b):
            numEdges = (numNodes * (numNodes - 1)) / 2
            return a * numEdges * np.log(numNodes) + b

        # Fit the model to the data
        mask = algoType == 'twiceAroundTheTree'
        popt, pcov = curve_fit(model_func, numNodes[mask], timeMinutes[mask])

        # Use the optimized parameters to plot the trend line
        x_range = np.linspace(min(numNodes[mask]), max(numNodes[mask]), 100)
        y_range = model_func(x_range, *popt)
        ax.plot(x_range, y_range, label='Twice Around the Tree Trend', color=color, alpha=0.2, linewidth=2)

# Calculate the y-coordinate for the 30-minute reference line
yReference = 30.0  # 30 minutes
yReferencePosition = (2 / 3) * yReference

# Add a reference line for 30 minutes with the custom dash pattern
ax.axhline(y=yReference, color='lightgrey', linestyle='--', label='Time Limit (30 mins)')

# Set plot labels and legend
ax.set_xlabel('Number of Nodes')
ax.set_ylabel('Time (minutes)')
ax.set_title('Number of Nodes vs. Time (with trend lines)')
ax.legend(loc='upper right')

# Set the y-axis limit to ensure the reference line is 2/3 of the way up
ax.set_ylim(0, max(timeMinutes) + 10)  # Adjust the upper limit as needed

# Show the plot
#plt.show()
plt.tight_layout()

# Save the plot to the ./plots directory
plt.savefig('./plots/numNodes_VS_time')




In [1861]:
import pandas as pd

def processDataFrame(df, filepath):
    # Add a new column "optimal" filled with "N/A" for all rows
    df['optimal'] = 'N/A'

    # Read the file line by line
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.split(':')
            if len(parts) == 2:
                word, integer = parts
                word = word.strip()
                integer = integer.strip()
                # Check if the word is present in the "name" column
                matchingRows = df[df['name'].str.contains(word)]
                if not matchingRows.empty:
                    # Set the "optimal" value for matching rows
                    df.loc[matchingRows.index, 'optimal'] = int(integer)

    # Add a new column "worseConstant" initialized with "N/A"
    df['worseConstant'] = 'N/A'

    # Calculate "worseConstant" where possible
    df['cost'] = pd.to_numeric(df['cost'], errors='coerce')  # Ensure "cost" is numeric
    df['optimal'] = pd.to_numeric(df['optimal'], errors='coerce')  # Ensure "optimal" is numeric

    # Calculate "worseConstant" only when cost is not 0 or "inf" and optimal is not N/A
    mask = (df['cost'] != 0) & (df['cost'] != float('inf')) & (~df['optimal'].isna())
    df.loc[mask, 'worseConstant'] = df['cost'] / df['optimal']

    return df


In [1862]:
filepath = '../data/optimal.txt'

results_df2 = processDataFrame(results_df, filepath)

results_df2_sorted = results_df2.sort_values(by='numNodes')

results_df2_sorted


,name,numNodes,time,memory,cost,cuts,leafReached,algoType,optimal,worseConstant
90,eil51,51,1800.000000,2536.0,1985.400000,49231427.0,True,BnB_original_cpp,426,4.660563
143,eil51,51,0.063197,32.0,480.875467,N/A,N/A,christofides,426,1.128816
185,eil51,51,1800.000000,2372.0,1313.470000,41336129.0,False,BnB_trivialSolution_cpp,426,3.083263
48,eil51,51,0.015195,44.0,640.901223,N/A,N/A,twiceAroundTheTree,426,1.504463
37,berlin52,52,0.017226,8.0,10116.014496,N/A,N/A,twiceAroundTheTree,7542,1.341291
...,...,...,...,...,...,...,...,...,...,...
16,pcb3038,3038,42.676046,8.0,196573.899222,N/A,N/A,twiceAroundTheTree,137694,1.427614
111,pcb3038,3038,1315.729642,8.0,154218.185880,N/A,N/A,christofides,137694,1.120007
146,fl3795,3795,1230.284508,8.0,31490.613233,N/A,N/A,christofides,28772,1.094488
51,fl3795,3795,64.046877,-1012.0,39072.832549,N/A,N/A,twiceAroundTheTree,28772,1.358016


In [1863]:
# Convert the 'worseConstant' column to numeric where possible
results_df2['worseConstant'] = pd.to_numeric(results_df2['worseConstant'], errors='coerce')

# Filter rows where 'worseConstant' is not NaN
filteredDf = results_df2.dropna(subset=['worseConstant'])

# Sort the DataFrame based on 'numNodes' in ascending order
filteredDf = filteredDf.sort_values(by='numNodes')

# Set the figure size
plt.figure(figsize=(6, 5))

subset = filteredDf[filteredDf['algoType'] == "twiceAroundTheTree"]
plt.scatter(subset['numNodes'], subset['worseConstant'], label="twiceAroundTheTree", c=algoColors["twiceAroundTheTree"], alpha=0.65)

subset = filteredDf[filteredDf['algoType'] == "christofides"]
plt.scatter(subset['numNodes'], subset['worseConstant'], label="christofides", c=algoColors["christofides"], alpha=0.65)

ax = plt.gca()

# Add a reference line for 2-aprox
ax.axhline(y=2, color='red', linestyle='--', label='2-Aproximation', alpha=0.65, linewidth=2.5)

# Add a reference line for 1.5-aprox
ax.axhline(y=1.5, color='purple', linestyle='--', label='1.5-Aproximation', alpha=0.65, linewidth=2.5)

# Add a reference line for optimal
ax.axhline(y=1, color='grey', linestyle='--', label='Optimal Solution', alpha=0.65, linewidth=2.5)
    
# Set labels and title
plt.xlabel('Number of Nodes')
plt.ylabel('How many Times Worse than Optimal')
plt.title('Number of Nodes vs. How many Times Worse than Optimal')

# Show the legend
ax.legend(loc='upper right')

# Show the plot
plt.tight_layout()
#plt.show()

# Save the plot to the ./plots directory
plt.savefig('./plots/numNodes_VS_worseConstant_ChrisAndTwice')

In [1864]:
# Group the DataFrame by 'algoType' and calculate the average memory
r = results_df2[results_df2['memory'] > 0]
averageMemory = r.groupby('algoType')['memory'].mean().reset_index()

# Rename the columns for clarity
averageMemory.columns = ['Algorithm Type', 'Average Memory']

# Display the resulting DataFrame
print(averageMemory)


            Algorithm Type  Average Memory
0         BnB_original_cpp     6956.875000
1  BnB_trivialSolution_cpp     3404.444444
2             christofides       33.694915
3       twiceAroundTheTree       35.728814


In [1865]:


# Calculate the minimum and maximum worseConstant for each algoType
min_values = results_df2.groupby('algoType')['worseConstant'].min()
max_values = results_df2.groupby('algoType')['worseConstant'].max()

# Print the results
for algoType, min_val, max_val in zip(min_values.index, min_values, max_values):
    print(f"Minimum worseConstant for {algoType}: {min_val}")
    print(f"Maximum worseConstant for {algoType}: {max_val}")

Minimum worseConstant for BnB_original_cpp: 4.66056338028169
Maximum worseConstant for BnB_original_cpp: 93.72369598475882
Minimum worseConstant for BnB_trivialSolution_cpp: 1.030791349809886
Maximum worseConstant for BnB_trivialSolution_cpp: 14.106668411753157
Minimum worseConstant for christofides: 1.0504638281314986
Maximum worseConstant for christofides: 1.2027483659415852
Minimum worseConstant for twiceAroundTheTree: 1.1943038188622066
Maximum worseConstant for twiceAroundTheTree: 1.5698109163978895


In [1866]:
# Convert the 'cuts' column to numeric where possible
results_df2['cuts'] = pd.to_numeric(results_df2['cuts'], errors='coerce')

# Filter for the two algoTypes and drop NaN in 'cuts'
filtered_df = results_df2[(results_df2['algoType'].isin(['BnB_original_cpp', 'BnB_trivialSolution_cpp'])) & results_df2['cuts'].notna()]

# Pivot the DataFrame
pivoted_df = filtered_df.pivot(index='name', columns='algoType', values='cuts')

# Drop rows where either algorithm's value is NaN
common_instances_df = pivoted_df.dropna()

# Merge with 'numNodes' for sorting
common_instances_df = common_instances_df.merge(results_df2[['name', 'numNodes']].drop_duplicates(), on='name')

# Sort based on 'numNodes'
common_instances_df = common_instances_df.sort_values(by='numNodes')

# Set the figure size
plt.figure(figsize=(10, 5))

# Adjust the bar width for separation and create the index for bar placement
bar_width = 0.35  # Keep the original bar width
gap = 0.05  # Define a small gap between the bars
index = np.arange(len(common_instances_df))

# Create side by side bar chart with a small space between them
plt.bar(index - bar_width/2 + gap/2, common_instances_df['BnB_original_cpp'], bar_width - gap, label='BnB_original_cpp')
plt.bar(index + bar_width/2 + gap/2, common_instances_df['BnB_trivialSolution_cpp'], bar_width - gap, label='BnB_trivialSolution_cpp', color='red')

# Set labels and title
plt.xlabel('TSP Instances in Increasing Number of Nodes')
plt.ylabel('Number of Cuts in Tens of Millions')
plt.title('TSP Instances (ordered by number of nodes) vs.Number of Cuts')
plt.xticks(index, common_instances_df['name'], rotation=60)

# Add legend
plt.legend()

# Show and save the plot
plt.tight_layout()
plt.savefig('./plots/barCuts_Bnb_comparison')

In [1867]:
# Convert the 'memory' column to numeric where possible
results_df2['memory'] = pd.to_numeric(results_df2['memory'], errors='coerce')

# Filter for the two algoTypes and drop NaN in 'memory'
filtered_df = results_df2[(results_df2['algoType'].isin(['BnB_original_cpp', 'BnB_trivialSolution_cpp'])) & results_df2['memory'].notna()]

# Pivot the DataFrame
pivoted_df = filtered_df.pivot(index='name', columns='algoType', values='memory')

# Drop rows where either algorithm's value is NaN
common_instances_df = pivoted_df.dropna()

# Merge with 'numNodes' for sorting
common_instances_df = common_instances_df.merge(results_df2[['name', 'numNodes']].drop_duplicates(), on='name')

# Sort based on 'numNodes'
common_instances_df = common_instances_df.sort_values(by='numNodes')

# Set the figure size
plt.figure(figsize=(10, 5))

# Adjust the bar width for separation and create the index for bar placement
bar_width = 0.4  # Keep the original bar width
gap = 0.08  # Define a small gap between the bars
index = np.arange(len(common_instances_df))

# Create side by side bar chart with a small space between them
plt.bar(index - bar_width/2 + gap/2, common_instances_df['BnB_original_cpp'], bar_width - gap, label='BnB_original_cpp')
plt.bar(index + bar_width/2 + gap/2, common_instances_df['BnB_trivialSolution_cpp'], bar_width - gap, label='BnB_trivialSolution_cpp', color='red')

# Set labels and title
plt.xlabel('TSP Instances in Increasing Number of Nodes')
plt.ylabel('Memory Used in KB')
plt.title('TSP Instances (ordered by number of nodes) vs. Memory in KB')
plt.xticks(index, common_instances_df['name'], rotation=60)

# Add legend
plt.legend()

# Show and save the plot
plt.tight_layout()
plt.savefig('./plots/barMemory_Bnb_comparison')